<a href="https://colab.research.google.com/github/agrigoridou/Recurrent-Neural-Networks/blob/main/%CE%92_Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Εγκατάσταση Απαιτούμενων Βιβλιοθηκών

In [1]:
!pip install torch torchvision torchaudio
!pip install torchtext
!pip install scikit-learn
!pip install pandas
!pip install tqdm

  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl (906.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 2.5.1 which is incompatible.
  Using cached torch-1.13.0-cp310-cp310-manylinux1_x86_64.whl.metadata (23 kB)
Using cached torch-1.13.0-cp310-cp310-manylinux1_x86_64.whl (890.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

In [2]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
with open("/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py", "r") as file:
    text = file.read()


In [4]:
with open("/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv", "r") as file:
    text = file.read()


In [5]:
with open("/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/test.csv", "r") as file:
    text = file.read()


In [6]:
# Διαβάζουμε το περιεχόμενο του αρχείου rnn.py
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Εμφανίζουμε το περιεχόμενο του αρχείου για να το δούμε
print(rnn_code)


# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('ag-news-classification-dataset/train.csv')
test_data = pd.read_csv('ag-news-classification-dataset/test.csv

In [7]:
# Update paths in the RNN code
updated_rnn_code = rnn_code.replace('ag-news-classification-dataset/train.csv', '/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
updated_rnn_code = updated_rnn_code.replace('ag-news-classification-dataset/test.csv', '/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/test.csv')

# Save the updated file to the correct location
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'w') as file:
    file.write(updated_rnn_code)

# Verify the updated code
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Print the updated content to verify
print(rnn_code)


# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

#Task 1: Εκτέλεση του κώδικα

In [8]:
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py



Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2136284



Epoch: 1
100% 112/112 [00:15<00:00,  7.04it/s]
Train Loss : 1.302
Valid Loss : 1.162 Valid Acc  : 0.566
Epoch: 2
100% 112/112 [00:16<00:00,  6.98it/s]
Train Loss : 1.082
Valid Loss : 1.022 Valid Acc  : 0.720
Epoch: 3
100% 112/112 [00:15<00:00,  7.00it/s]
Train Loss : 0.975
Valid Loss : 0.962 Valid Acc  : 0.779
Epoch: 4
100% 112/112 [00:16<00:00,  6.86it/s]
Train Loss : 0.931
Valid Loss : 0.936 Valid Acc  : 0.804
Epoch: 5
100% 112/112 [00:28<00:00,  3.91it/s]
Train Loss : 0.907
Valid Loss : 0.918 Valid Acc  : 0.825
Epoch: 6
100% 112/112 [00:16<00:00,  6.81it/s]
Train Loss : 0.888
Valid Loss : 0.909 Valid Acc  : 0.831
Epoch: 7
100% 112/112 [00:16<00:00,  6.88it/s]
Train Loss : 0.876
Valid Loss : 0.894 Valid Acc  : 0.848
Epoch: 8
100% 112/112 [00:16<00:00,  6.91it/s]
Train Loss : 0.866
Valid Loss : 0



```
(i) Ακρίβεια στο Test Set: 86.5%
(ii) Ακρίβεια στο Validation Set: 87.0%
(iii) Πλήθος Παραμέτρων: 2136284
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης ανά Εποχή: περίπου 16 δευτερόλεπτα ανά εποχή.

```



#Task 2: Δι-κατευθυντήριο RNN

Δι-κατευθυντήριο RNN: Στην τρέχουσα εκδοχή του κώδικα, το RNN layer είναι μονοκατευθυντικό (unidirectional). Για να το κάνουμε δι-κατευθυντήριο, πρέπει να προσθέσουμε το bidirectional=True στον ορισμό του RNN layer. Επίσης, επειδή το δι-κατευθυντήριο RNN συνενώνει τις εξόδους των δύο κατευθύνσεων, το μέγεθος του διανύσματος εξόδου θα διπλασιαστεί, οπότε θα πρέπει να αυξήσουμε το μέγεθος της εξόδου του γραμμικού (linear) layer για να το αντιστοιχίσουμε με το νέο μέγεθος του διανύσματος εξόδου.

Αλλαγές στον κώδικα:

*   Τροποποιούμε την κλάση του RNN για να είναι δι-κατευθυντική.
*   Διπλασιάζουμε το μέγεθος των εξόδων του RNN στον γραμμικό (linear) layer.




In [9]:
# Διαβάζουμε το περιεχόμενο του αρχείου rnn.py
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Εμφανίζουμε το περιεχόμενο του αρχείου για να το δούμε
print(rnn_code)


# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

In [10]:
# Αντικαθιστούμε το RNN με bidirectional RNN και προσαρμόζουμε το γραμμικό layer
updated_rnn_code = rnn_code.replace(
    'self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)',
    'self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)'
)
updated_rnn_code = updated_rnn_code.replace(
    'self.linear = nn.Linear(hidden_dim, output_dim)',
    'self.linear = nn.Linear(hidden_dim * 2, output_dim)'  # Για διπλασιασμένο hidden_dim λόγω bidirectional
)

# Αποθήκευση του τροποποιημένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'w') as file:
    file.write(updated_rnn_code)

# Επαλήθευση του ενημερωμένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Εκτύπωση του ενημερωμένου κώδικα για επαλήθευση
print(rnn_code)


# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

In [11]:
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py




Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=4, bias=True)
)
Total parameters:  2147164



Epoch: 1
100% 112/112 [00:23<00:00,  4.78it/s]
Train Loss : 1.302
Valid Loss : 1.161 Valid Acc  : 0.586
Epoch: 2
100% 112/112 [00:24<00:00,  4.66it/s]
Train Loss : 1.076
Valid Loss : 1.015 Valid Acc  : 0.731
Epoch: 3
100% 112/112 [00:24<00:00,  4.51it/s]
Train Loss : 0.976
Valid Loss : 0.963 Valid Acc  : 0.782
Epoch: 4
100% 112/112 [00:23<00:00,  4.82it/s]
Train Loss : 0.933
Valid Loss : 0.937 Valid Acc  : 0.807
Epoch: 5
100% 112/112 [00:24<00:00,  4.54it/s]
Train Loss : 0.907
Valid Loss : 0.922 Valid Acc  : 0.819
Epoch: 6
100% 112/112 [00:24<00:00,  4.51it/s]
Train Loss : 0.891
Valid Loss : 0.913 Valid Acc  : 0.829
Epoch: 7
100% 112/112 [00:23<00:00,  4.79it/s]
Train Loss : 0.880
Valid Loss : 0.897 Valid Acc  : 0.845
Epoch: 8
100% 112/112 [00:24<00:00,  4.50it/s]
Train Loss :



```
(i) Ακρίβεια Ταξινόμησης στο Test Set: 85.3%
(ii) Μέση Ακρίβεια Ταξινόμησης στο Validation Set: 86.6%
(iii) Πλήθος Παραμέτρων Μοντέλου: 2147164
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης Ανά Εποχή: 25 δευτερόλεπτα

```



# Task 3: Διπλό στρώμα δι-κατευθυντήριου RNN

##1. Αλλαγή στην αρχικοποίηση του RNN:


*   Αυξάνουμε τον αριθμό των επιπέδων του RNN από 1 σε 2. Αυτό γίνεται με την παράμετρο num_layers=2.
*   Ρυθμίζουμε το RNN ώστε να είναι δι-κατευθυντήριο με την παράμετρο bidirectional=True.



##2. Αλλαγή στον γραμμικό (linear) layer:

Επειδή το δι-κατευθυντήριο RNN διπλασιάζει το μέγεθος του hidden state, και

*   τώρα έχουμε δύο στρώματα, το μέγεθος του hidden state θα είναι 2 * hidden_dim (διπλό λόγω δι-κατευθυντηριότητας και λόγω δύο στρωμάτων). Άρα το hidden_dim πρέπει να πολλαπλασιαστεί με 2 ακόμα, δηλαδή το νέο μέγεθος του hidden state είναι hidden_dim * 2 * 2 λόγω δύο επιπέδων και δι-κατευθυντηριότητας.



In [12]:
# Ενημερώνουμε το RNN για να είναι δύο στρώματα δι-κατευθυντήριου
updated_rnn_code = rnn_code.replace(
    'self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)',
    'self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=2, batch_first=True, bidirectional=True)'
)

# Ενημερώνουμε το γραμμικό layer για να έχει το σωστό μέγεθος εξόδου
updated_rnn_code = updated_rnn_code.replace(
    'logits = self.linear(output[:, -1])',
    'logits = self.linear(output[:, -1, :])'  # Για δύο στρώματα δι-κατευθυντήριου RNN, το μέγεθος είναι hidden_dim * 2 * 2
)

# Αποθήκευση του τροποποιημένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'w') as file:
    file.write(updated_rnn_code)

# Επαλήθευση του ενημερωμένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Εκτύπωση του ενημερωμένου κώδικα για επαλήθευση
print(rnn_code)


# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

In [13]:
# Εκτέλεση του τροποποιημένου κώδικα
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py


Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, num_layers=2, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=4, bias=True)
)
Total parameters:  2171996



Epoch: 1
100% 112/112 [00:38<00:00,  2.92it/s]
Train Loss : 1.255
Valid Loss : 1.103 Valid Acc  : 0.634
Epoch: 2
100% 112/112 [00:38<00:00,  2.89it/s]
Train Loss : 1.039
Valid Loss : 0.996 Valid Acc  : 0.744
Epoch: 3
100% 112/112 [00:37<00:00,  3.02it/s]
Train Loss : 0.970
Valid Loss : 0.963 Valid Acc  : 0.776
Epoch: 4
100% 112/112 [00:38<00:00,  2.90it/s]
Train Loss : 0.937
Valid Loss : 0.939 Valid Acc  : 0.804
Epoch: 5
100% 112/112 [00:37<00:00,  3.00it/s]
Train Loss : 0.916
Valid Loss : 0.925 Valid Acc  : 0.816
Epoch: 6
100% 112/112 [00:38<00:00,  2.94it/s]
Train Loss : 0.905
Valid Loss : 0.945 Valid Acc  : 0.796
Epoch: 7
100% 112/112 [00:38<00:00,  2.88it/s]
Train Loss : 0.897
Valid Loss : 0.926 Valid Acc  : 0.814
Epoch: 8
100% 112/112 [00:37<00:00,  3.00it/s



```
(i) Ακρίβεια Ταξινόμησης στο Test Set: 85.8%
(ii) Μέση Ακρίβεια Ταξινόμησης στο Validation Set: 86.0%
(iii) Πλήθος Παραμέτρων Μοντέλου: 2117996
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης Ανά Εποχή: 38 δευτερόλεπτα
```



#Task 4: Χρήση LSTM

In [36]:
# Verify the updated code
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Print the updated content to verify
print(rnn_code)

# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('ag-news-classification-dataset/train.csv')
test_data = pd.read_csv('ag-news-classification-dataset/test.csv

#Βήμα 1: Αντικατάσταση του RNN με LSTM

In [37]:
# Update paths in the RNN code
updated_rnn_code = rnn_code.replace('ag-news-classification-dataset/train.csv', '/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
updated_rnn_code = updated_rnn_code.replace('ag-news-classification-dataset/test.csv', '/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/test.csv')

# Save the updated file to the correct location
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'w') as file:
    file.write(updated_rnn_code)

# Verify the updated code
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Print the updated content to verify
print(rnn_code)


# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

Θα αντικαταστήσουμε την αρχική χρήση του RNN με LSTM και θα κάνουμε τις απαραίτητες προσαρμογές στον κώδικα για να δουλέψει σωστά με το LSTM. Παρακάτω παρατίθεται το κομμάτι κώδικα που πρέπει να τροποποιηθεί για να χρησιμοποιηθεί LSTM αντί για RNN.

Αλλαγές στον κώδικα:

*   Αντικαθιστούμε το nn.RNN με nn.LSTM.
*   Επειδή το LSTM έχει δύο εξόδους, h_n και c_n, πρέπει να προσαρμόσουμε την εξαγωγή της εξόδου για να χρησιμοποιήσουμε μόνο την πρώτη έξοδο (που είναι το h_n).





In [38]:
# Αντικατάσταση του RNN με LSTM
updated_rnn_code = rnn_code.replace(
    'self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)',
    'self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)'
)

# Αποθήκευση του τροποποιημένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'w') as file:
    file.write(updated_rnn_code)

# Επαλήθευση του ενημερωμένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Εκτύπωση του ενημερωμένου κώδικα για επαλήθευση
print(rnn_code)

# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

In [39]:
# Εκτέλεση του τροποποιημένου κώδικα
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py


Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): LSTM(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2168156



Epoch: 1
100% 112/112 [00:35<00:00,  3.17it/s]
Train Loss : 1.256
Valid Loss : 1.046 Valid Acc  : 0.705
Epoch: 2
100% 112/112 [00:33<00:00,  3.31it/s]
Train Loss : 0.976
Valid Loss : 0.936 Valid Acc  : 0.809
Epoch: 3
100% 112/112 [00:33<00:00,  3.35it/s]
Train Loss : 0.911
Valid Loss : 0.903 Valid Acc  : 0.841
Epoch: 4
100% 112/112 [00:34<00:00,  3.24it/s]
Train Loss : 0.882
Valid Loss : 0.887 Valid Acc  : 0.859
Epoch: 5
100% 112/112 [00:34<00:00,  3.24it/s]
Train Loss : 0.864
Valid Loss : 0.880 Valid Acc  : 0.862
Epoch: 6
100% 112/112 [00:33<00:00,  3.34it/s]
Train Loss : 0.851
Valid Loss : 0.876 Valid Acc  : 0.866
Epoch: 7
100% 112/112 [00:34<00:00,  3.28it/s]
Train Loss : 0.843
Valid Loss : 0.874 Valid Acc  : 0.869
Epoch: 8
100% 112/112 [00:34<00:00,  3.27it/s]
Train Loss : 0.836
Valid Loss : 



```
(i) Ακρίβεια Ταξινόμησης στο Test Set: 88.0%
(ii) Μέση Ακρίβεια Ταξινόμησης στο Validation Set: 87.8%
(iii) Πλήθος Παραμέτρων Μοντέλου: 2168156
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης Ανά Εποχή: 33.5 δευτερόλεπτα
```



#Βήμα 2: Ενημέρωση για Δι-κατευθυντήριο LSTM

Όπως κάναμε για το RNN, θα προσθέσουμε την παράμετρο bidirectional=True για να το κάνουμε δι-κατευθυντήριο. Αυτό θα διπλασιάσει το μέγεθος του hidden state.

In [40]:
# Ενημέρωση για Δι-κατευθυντήριο LSTM
updated_rnn_code = updated_rnn_code.replace(
    'self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)',
    'self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)'
)

# Ενημέρωση του γραμμικού layer για διπλασιασμένο μέγεθος εξόδου
updated_rnn_code = updated_rnn_code.replace(
    'self.linear = nn.Linear(hidden_dim, output_dim)',
    'self.linear = nn.Linear(hidden_dim * 2, output_dim)'
)


# Αποθήκευση του τροποποιημένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'w') as file:
    file.write(updated_rnn_code)

# Επαλήθευση του ενημερωμένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Εκτύπωση του ενημερωμένου κώδικα για επαλήθευση
print(rnn_code)

# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

In [41]:
# Εκτέλεση του τροποποιημένου κώδικα
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py


Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): LSTM(100, 64, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=4, bias=True)
)
Total parameters:  2210908



Epoch: 1
100% 112/112 [01:00<00:00,  1.85it/s]
Train Loss : 1.251
Valid Loss : 1.046 Valid Acc  : 0.703
Epoch: 2
100% 112/112 [01:00<00:00,  1.87it/s]
Train Loss : 0.980
Valid Loss : 0.945 Valid Acc  : 0.801
Epoch: 3
100% 112/112 [00:59<00:00,  1.88it/s]
Train Loss : 0.914
Valid Loss : 0.913 Valid Acc  : 0.832
Epoch: 4
100% 112/112 [00:59<00:00,  1.89it/s]
Train Loss : 0.883
Valid Loss : 0.898 Valid Acc  : 0.848
Epoch: 5
100% 112/112 [00:58<00:00,  1.91it/s]
Train Loss : 0.865
Valid Loss : 0.890 Valid Acc  : 0.852
Epoch: 6
100% 112/112 [01:02<00:00,  1.79it/s]
Train Loss : 0.852
Valid Loss : 0.882 Valid Acc  : 0.860
Epoch: 7
100% 112/112 [01:01<00:00,  1.82it/s]
Train Loss : 0.843
Valid Loss : 0.877 Valid Acc  : 0.865
Epoch: 8
100% 112/112 [01:02<00:00,  1.80it/s]
Train Loss 



```
(i) Ακρίβεια Ταξινόμησης στο Test Set: 87.9%
(ii) Μέση Ακρίβεια Ταξινόμησης στο Validation Set: 87.8%
(iii) Πλήθος Παραμέτρων Μοντέλου: 2,210,908
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης Ανά Εποχή: 61.0 δευτερόλεπτα
```



#Βήμα 3: Ενημέρωση για Διπλό Στρώμα Δι-κατευθυντήριου LSTM

Όπως και με το RNN, θα προσθέσουμε και ένα δεύτερο επίπεδο στο LSTM και θα διπλασιάσουμε το μέγεθος του hidden state.

In [42]:
# Ενημέρωση για Διπλό Στρώμα Δι-κατευθυντήριο LSTM
updated_rnn_code = updated_rnn_code.replace(
    'self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)',
    'self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=2, batch_first=True, bidirectional=True)'
)

# Ενημέρωση του γραμμικού layer για διπλό στρώμα
updated_rnn_code = updated_rnn_code.replace(
    'logits = self.linear(output[:, -1])',
    'logits = self.linear(output[:, -1, :])'  # Για δύο στρώματα δι-κατευθυντήριου RNN, το μέγεθος είναι hidden_dim * 2 * 2
)


# Αποθήκευση του τροποποιημένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'w') as file:
    file.write(updated_rnn_code)

# Επαλήθευση του ενημερωμένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Εκτύπωση του ενημερωμένου κώδικα για επαλήθευση
print(rnn_code)


# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

In [43]:
# Εκτέλεση του τροποποιημένου κώδικα
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py


Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): LSTM(100, 64, num_layers=2, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=128, out_features=4, bias=True)
)
Total parameters:  2310236



Epoch: 1
100% 112/112 [01:52<00:00,  1.00s/it]
Train Loss : 1.202
Valid Loss : 1.025 Valid Acc  : 0.713
Epoch: 2
100% 112/112 [01:50<00:00,  1.01it/s]
Train Loss : 0.956
Valid Loss : 0.928 Valid Acc  : 0.813
Epoch: 3
100% 112/112 [01:53<00:00,  1.01s/it]
Train Loss : 0.897
Valid Loss : 0.893 Valid Acc  : 0.847
Epoch: 4
100% 112/112 [01:52<00:00,  1.01s/it]
Train Loss : 0.874
Valid Loss : 0.883 Valid Acc  : 0.859
Epoch: 5
100% 112/112 [01:52<00:00,  1.01s/it]
Train Loss : 0.856
Valid Loss : 0.875 Valid Acc  : 0.866
Epoch: 6
100% 112/112 [01:55<00:00,  1.03s/it]
Train Loss : 0.845
Valid Loss : 0.868 Valid Acc  : 0.875
Epoch: 7
100% 112/112 [01:51<00:00,  1.01it/s]
Train Loss : 0.838
Valid Loss : 0.872 Valid Acc  : 0.869
Epoch: 8
100% 112/112 [01:51<00:00,  1.01it/



```
(i) Ακρίβεια Ταξινόμησης στο Test Set: 88.3%
(ii) Μέση Ακρίβεια Ταξινόμησης στο Validation Set: 88.5%
(iii) Πλήθος Παραμέτρων Μοντέλου: 2,310,236
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης Ανά Εποχή: 1 λεπτό και 51 δευτερόλεπτα
```



# Task 5:

In [61]:
import pandas as pd

# Δημιουργία του DataFrame με τα δεδομένα
data = {
    'Μοντέλο': ['1RNN', '1Bi-RNN', '2Bi-RNN', '1LSTM', '1Bi-LSTM', '2Bi-LSTM'],
    'Accuracy (Test Set)': ['86.5%', '85.3%', '85.8%', '88.0%', '87.9%', '88.3%'],
    'Accuracy (Validation Set)': ['87.0%', '86.6%', '86.0%', '87.8%', '87.8%', '88.5%'],
    'Πλήθος Παραμέτρων': [2136284, 2147164, 2117996, 2168156, 2210908, 2310236],
    'Χρόνος Εκπαίδευσης ανά Εποχή': ['16 δευτερόλεπτα', '25 δευτερόλεπτα', '38 δευτερόλεπτα',
                                    '33.5 δευτερόλεπτα', '61.0 δευτερόλεπτα', '1 λεπτό 51 δευτερόλεπτα']
}

# Δημιουργία DataFrame
df = pd.DataFrame(data)

# Εμφάνιση του πίνακα
df


,Μοντέλο,Accuracy (Test Set),Accuracy (Validation Set),Πλήθος Παραμέτρων,Χρόνος Εκπαίδευσης ανά Εποχή
0,1RNN,86.5%,87.0%,2136284,16 δευτερόλεπτα
1,1Bi-RNN,85.3%,86.6%,2147164,25 δευτερόλεπτα
2,2Bi-RNN,85.8%,86.0%,2117996,38 δευτερόλεπτα
3,1LSTM,88.0%,87.8%,2168156,33.5 δευτερόλεπτα
4,1Bi-LSTM,87.9%,87.8%,2210908,61.0 δευτερόλεπτα
5,2Bi-LSTM,88.3%,88.5%,2310236,1 λεπτό 51 δευτερόλεπτα


|index|Μοντέλο|Accuracy \(Test Set\)|Accuracy \(Validation Set\)|Πλήθος Παραμέτρων|Χρόνος Εκπαίδευσης ανά Εποχή|
|---|---|---|---|---|---|
|0|1RNN|86\.5%|87\.0%|2136284|16 δευτερόλεπτα|
|1|1Bi-RNN|85\.3%|86\.6%|2147164|25 δευτερόλεπτα|
|2|2Bi-RNN|85\.8%|86\.0%|2117996|38 δευτερόλεπτα|
|3|1LSTM|88\.0%|87\.8%|2168156|33\.5 δευτερόλεπτα|
|4|1Bi-LSTM|87\.9%|87\.8%|2210908|61\.0 δευτερόλεπτα|
|5|2Bi-LSTM|88\.3%|88\.5%|2310236|1 λεπτό 51 δευτερόλεπτα|

##1. Πόσο επηρεάζεται η επίδοση του δικτύου RNN/LSTM από την επιλογή του μοντέλου;

Η επίδοση του δικτύου επηρεάζεται σημαντικά από την επιλογή του μοντέλου. Τα αποτελέσματα δείχνουν ότι τα LSTM μοντέλα (και ιδιαίτερα τα Bi-LSTM) υπερέχουν σε σχέση με τα απλά RNN μοντέλα:


*   Τα LSTM μοντέλα (1LSTM, 1Bi-LSTM, 2Bi-LSTM) εμφανίζουν υψηλότερη ακρίβεια στο test set σε σχέση με τα RNN μοντέλα (1RNN, 1Bi-RNN, 2Bi-RNN). Για παράδειγμα, το 2Bi-LSTM πετυχαίνει 88.3% στην ακρίβεια του test set, ενώ το 2Bi-RNN φτάνει μόνο το 85.8%.
*   Τα Bi-LSTM μοντέλα φαίνεται να είναι πιο ισχυρά στην κατανόηση της σειράς και της εξάρτησης στα δεδομένα, κάτι που οδηγεί σε καλύτερη απόδοση.





##2. Πόσο επηρεάζεται η πολυπλοκότητα (παράμετροι, χρόνος εκπαίδευσης) από την επιλογή του μοντέλου;

Η πολυπλοκότητα (τόσο ο αριθμός παραμέτρων όσο και ο χρόνος εκπαίδευσης) αυξάνεται καθώς μεταβαίνουμε από τα RNN στα LSTM μοντέλα και από τα 1RNN/1LSTM στα 2Bi-RNN/2Bi-LSTM:


*   Πλήθος παραμέτρων: Τα Bi-LSTM μοντέλα έχουν περισσότερες παραμέτρους. Για παράδειγμα, το 2Bi-LSTM έχει 2,310,236 παραμέτρους, ενώ το 1RNN έχει 2,136,284 παραμέτρους.
*   Χρόνος εκπαίδευσης: Ο χρόνος εκπαίδευσης ανά εποχή είναι μεγαλύτερος για τα Bi-LSTM μοντέλα. Για παράδειγμα, το 2Bi-LSTM χρειάζεται 1 λεπτό και 51 δευτερόλεπτα ανά εποχή, ενώ το 1RNN χρειάζεται μόνο 16 δευτερόλεπτα ανά εποχή.





Αυτό υποδηλώνει ότι η χρήση των Bi-LSTM μοντέλων αυξάνει την πολυπλοκότητα, αλλά ταυτόχρονα βελτιώνει την επίδοση.

##3. Διαφέρει σημαντικά η επίδοση στο test set και στο validation set;

Η διαφορά μεταξύ test set και validation set είναι μικρή σε όλα τα μοντέλα, τόσο για τα RNN όσο και για τα LSTM:




*   Η διαφορά στην ακρίβεια κυμαίνεται γύρω από το 0.3% - 0.5% σε όλα τα μοντέλα.
*   Αυτή η μικρή διαφορά υποδηλώνει ότι τα μοντέλα γενικά γενικεύουν καλά στα δεδομένα του test set, χωρίς υπερβολική εξειδίκευση στα δεδομένα εκπαίδευσης.



##4. Ποιο είδος μοντέλου φαίνεται να υπερέχει, το RNN ή το LSTM;

Το LSTM υπερέχει ξεκάθαρα σε σχέση με το RNN όσον αφορά την ακρίβεια και την απόδοση:


*   Τα LSTM μοντέλα (1LSTM, 1Bi-LSTM, 2Bi-LSTM) εμφανίζουν σημαντικά καλύτερη επίδοση σε σύγκριση με τα RNN μοντέλα.
*   Ο λόγος είναι ότι τα LSTM είναι ικανά να διατηρούν μακροπρόθεσμες εξαρτήσεις και να επιλύουν τα προβλήματα της εξαφάνισης του βαθμού (vanishing gradient), κάτι που τους επιτρέπει να έχουν καλύτερη απόδοση στις εργασίες ταξινόμησης.




# Task 6:

In [52]:
# Verify the updated code
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Print the updated content to verify
print(rnn_code)

# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('ag-news-classification-dataset/train.csv')
test_data = pd.read_csv('ag-news-classification-dataset/test.csv

In [53]:
# Update paths in the RNN code
updated_rnn_code = rnn_code.replace('ag-news-classification-dataset/train.csv', '/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
updated_rnn_code = updated_rnn_code.replace('ag-news-classification-dataset/test.csv', '/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/test.csv')

# Save the updated file to the correct location
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'w') as file:
    file.write(updated_rnn_code)

# Verify the updated code
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Print the updated content to verify
print(rnn_code)


# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

##Βήμα 1: Ενημέρωση της παραμέτρου MAX_WORDS στα μοντέλα

In [54]:
# Αλλαγή της παραμέτρου MAX_WORDS
updated_rnn_code = rnn_code.replace('MAX_WORDS = 25', 'MAX_WORDS = 50')


# Αποθήκευση του τροποποιημένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'w') as file:
    file.write(updated_rnn_code)

# Επαλήθευση του ενημερωμένου κώδικα
with open('/content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py', 'r') as file:
    rnn_code = file.read()

# Εκτύπωση του ενημερωμένου κώδικα για επαλήθευση
print(rnn_code)


# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# HYPER-PARAMETERS
MAX_WORDS = 50
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

######################################################################
# Read dataset files 
# ------------------


train_data = pd.read_csv('/content/drive/MyDrive/RecurrentNeuralNetworks/ag-news-classification-dataset/train.csv')
test_data = pd.re

##Βήμα 2: Εκτέλεση των μοντέλων RNN (απλό και δι-κατευθυντήριο)

###Εκτέλεση για απλό RNN:

In [55]:
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py



Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2136284



Epoch: 1
100% 112/112 [00:25<00:00,  4.48it/s]
Train Loss : 1.380
Valid Loss : 1.346 Valid Acc  : 0.346
Epoch: 2
100% 112/112 [00:25<00:00,  4.42it/s]
Train Loss : 1.329
Valid Loss : 1.312 Valid Acc  : 0.397
Epoch: 3
100% 112/112 [00:25<00:00,  4.33it/s]
Train Loss : 1.302
Valid Loss : 1.311 Valid Acc  : 0.395
Epoch: 4
100% 112/112 [00:25<00:00,  4.37it/s]
Train Loss : 1.324
Valid Loss : 1.312 Valid Acc  : 0.386
Epoch: 5
100% 112/112 [00:24<00:00,  4.55it/s]
Train Loss : 1.288
Valid Loss : 1.311 Valid Acc  : 0.361
Epoch: 6
100% 112/112 [00:26<00:00,  4.29it/s]
Train Loss : 1.294
Valid Loss : 1.301 Valid Acc  : 0.396
Epoch: 7
100% 112/112 [00:25<00:00,  4.33it/s]
Train Loss : 1.304
Valid Loss : 1.296 Valid Acc  : 0.406
Epoch: 8
100% 112/112 [00:26<00:00,  4.28it/s]
Train Loss : 1.281
Valid Loss : 1



```
(i) Ακρίβεια Ταξινόμησης στο Test Set: 37.8%
(ii) Μέση Ακρίβεια Ταξινόμησης στο Validation Set: 38.2%
(iii) Πλήθος Παραμέτρων Μοντέλου: 2,136,284
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης Ανά Εποχή: 25 δευτερόλεπτα
```



###Τροποποίηση για δι-κατευθυντήριο RNN:

In [56]:
updated_rnn_code = updated_rnn_code.replace(
    'self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)',
    'self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)'
)
updated_rnn_code = updated_rnn_code.replace(
    'self.linear = nn.Linear(hidden_dim, output_dim)',
    'self.linear = nn.Linear(hidden_dim * 2, output_dim)'
)


In [57]:
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py



Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2136284



Epoch: 1
100% 112/112 [00:26<00:00,  4.25it/s]
Train Loss : 1.383
Valid Loss : 1.379 Valid Acc  : 0.295
Epoch: 2
100% 112/112 [00:25<00:00,  4.44it/s]
Train Loss : 1.339
Valid Loss : 1.294 Valid Acc  : 0.412
Epoch: 3
100% 112/112 [00:28<00:00,  3.93it/s]
Train Loss : 1.303
Valid Loss : 1.298 Valid Acc  : 0.409
Epoch: 4
100% 112/112 [00:24<00:00,  4.49it/s]
Train Loss : 1.286
Valid Loss : 1.285 Valid Acc  : 0.427
Epoch: 5
100% 112/112 [00:25<00:00,  4.40it/s]
Train Loss : 1.287
Valid Loss : 1.349 Valid Acc  : 0.336
Epoch: 6
100% 112/112 [00:25<00:00,  4.34it/s]
Train Loss : 1.277
Valid Loss : 1.256 Valid Acc  : 0.451
Epoch: 7
100% 112/112 [00:26<00:00,  4.30it/s]
Train Loss : 1.235
Valid Loss : 1.235 Valid Acc  : 0.460
Epoch: 8
100% 112/112 [00:26<00:00,  4.22it/s]
Train Loss : 1.216
Valid Loss : 1



```
(i) Ακρίβεια Ταξινόμησης στο Test Set: 39.1%
(ii) Μέση Ακρίβεια Ταξινόμησης στο Validation Set: 39.8%
(iii) Πλήθος Παραμέτρων Μοντέλου: 2,136,284
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης Ανά Εποχή: 26 δευτερόλεπτα
```



##Βήμα 3: Εκτέλεση των μοντέλων LSTM (απλό και δι-κατευθυντήριο)

Τροποποίηση για απλό LSTM: Αλλάξτε το RNN σε LSTM στο αρχείο:

In [58]:
updated_rnn_code = updated_rnn_code.replace('self.rnn = nn.RNN', 'self.rnn = nn.LSTM')


Εκτέλεση για απλό LSTM:

In [59]:
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py



Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2136284



Epoch: 1
100% 112/112 [00:26<00:00,  4.28it/s]
Train Loss : 1.383
Valid Loss : 1.380 Valid Acc  : 0.279
Epoch: 2
100% 112/112 [00:29<00:00,  3.84it/s]
Train Loss : 1.354
Valid Loss : 1.360 Valid Acc  : 0.325
Epoch: 3
100% 112/112 [00:37<00:00,  3.02it/s]
Train Loss : 1.369
Valid Loss : 1.361 Valid Acc  : 0.312
Epoch: 4
100% 112/112 [00:30<00:00,  3.61it/s]
Train Loss : 1.367
Valid Loss : 1.358 Valid Acc  : 0.328
Epoch: 5
100% 112/112 [00:26<00:00,  4.20it/s]
Train Loss : 1.363
Valid Loss : 1.355 Valid Acc  : 0.333
Epoch: 6
100% 112/112 [00:25<00:00,  4.40it/s]
Train Loss : 1.357
Valid Loss : 1.359 Valid Acc  : 0.328
Epoch: 7
100% 112/112 [00:25<00:00,  4.41it/s]
Train Loss : 1.361
Valid Loss : 1.358 Valid Acc  : 0.330
Epoch: 8
100% 112/112 [00:25<00:00,  4.32it/s]
Train Loss : 1.356
Valid Loss : 1



```
(i) Ακρίβεια Ταξινόμησης στο Test Set: 31.2%
(ii) Μέση Ακρίβεια Ταξινόμησης στο Validation Set: 31.9%
(iii) Πλήθος Παραμέτρων Μοντέλου: 2,136,284
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης Ανά Εποχή: 29.2 δευτερόλεπτα
```



Τροποποίηση για δι-κατευθυντήριο LSTM:

In [62]:
updated_rnn_code = updated_rnn_code.replace(
    'self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)',
    'self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)'
)
updated_rnn_code = updated_rnn_code.replace(
    'self.linear = nn.Linear(hidden_dim, output_dim)',
    'self.linear = nn.Linear(hidden_dim * 2, output_dim)'
)


In [63]:
!python3 /content/drive/MyDrive/RecurrentNeuralNetworks/rnn.py



Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2136284



Epoch: 1
100% 112/112 [00:26<00:00,  4.24it/s]
Train Loss : 1.380
Valid Loss : 1.351 Valid Acc  : 0.331
Epoch: 2
100% 112/112 [00:24<00:00,  4.52it/s]
Train Loss : 1.360
Valid Loss : 1.363 Valid Acc  : 0.314
Epoch: 3
100% 112/112 [00:24<00:00,  4.50it/s]
Train Loss : 1.355
Valid Loss : 1.361 Valid Acc  : 0.328
Epoch: 4
100% 112/112 [00:26<00:00,  4.21it/s]
Train Loss : 1.333
Valid Loss : 1.315 Valid Acc  : 0.384
Epoch: 5
100% 112/112 [00:25<00:00,  4.43it/s]
Train Loss : 1.372
Valid Loss : 1.370 Valid Acc  : 0.297
Epoch: 6
100% 112/112 [00:24<00:00,  4.62it/s]
Train Loss : 1.311
Valid Loss : 1.260 Valid Acc  : 0.428
Epoch: 7
100% 112/112 [00:25<00:00,  4.35it/s]
Train Loss : 1.323
Valid Loss : 1.353 Valid Acc  : 0.341
Epoch: 8
100% 112/112 [00:25<00:00,  4.34it/s]
Train Loss : 1.327
Valid Loss : 1



```
(i) Ακρίβεια Ταξινόμησης στο Test Set: 59.9%
(ii) Μέση Ακρίβεια Ταξινόμησης στο Validation Set: 59.4%
(iii) Πλήθος Παραμέτρων Μοντέλου: 2,136,284
(iv) Μέσο Χρονικό Κόστος Εκπαίδευσης Ανά Εποχή: 25.3 δευτερόλεπτα
```



Η αλλαγή από απλό RNN σε δι-κατευθυντικό RNN ή LSTM βελτιώνει την απόδοση, καθώς επιτρέπει στο μοντέλο να κατανοήσει καλύτερα τις εξαρτήσεις στα δεδομένα, ειδικά σε ακολουθίες με μακροχρόνιες εξαρτήσεις. Η διπλή κατεύθυνση (bidirectional) αυξάνει την ακρίβεια, όπως φαίνεται από τα αποτελέσματα (π.χ., από 37.8% σε 39.1% για το RNN). Τα LSTM προσφέρουν καλύτερη απόδοση από τα RNN, ειδικά με την προσθήκη της διπλής κατεύθυνσης, αυξάνοντας την ακρίβεια σημαντικά (από 31.2% σε 59.9%).

Ωστόσο, αυτή η βελτίωση στην απόδοση συνοδεύεται από αυξημένη πολυπλοκότητα, καθώς απαιτούν περισσότερους υπολογιστικούς πόρους και μεγαλύτερο χρόνο εκπαίδευσης.

Αν παρατηρήσουμε τα αποτελέσματα:


*   RNN (απλό): Προσφέρει σχετικά χαμηλή ακρίβεια (37.8%), αλλά είναι το λιγότερο υπολογιστικά απαιτητικό.
*   Δι-κατευθυντικό RNN: Η χρήση διπλής κατεύθυνσης αυξάνει την ακρίβεια (39.1%), αν και παραμένει σχετικά χαμηλή.
*   LSTM (απλό): Εντυπωσιακή πτώση στην απόδοση με ακρίβεια 31.2%. Παρόλο που τα LSTM θεωρούνται καλύτερα για μεγάλες ακολουθίες, φαίνεται ότι το μοντέλο δεν αποδίδει τόσο καλά στο συγκεκριμένο πρόβλημα.
*   Δι-κατευθυντικό LSTM: Το καλύτερο αποτέλεσμα με σημαντική αύξηση στην ακρίβεια (59.9%), επιβεβαιώνοντας ότι η χρήση LSTM σε συνδυασμό με τη διπλή κατεύθυνση είναι το πιο ισχυρό μοντέλο.







# Task 7:

# Task 8:

# Task 9:

# Task 10: